In [1]:
# %matplotlib inline 
import pandas as pd
import sklearn as sk
from matplotlib import pylab as plt
import numpy as np
from statsmodels.tsa import stattools as stat

In [2]:
data = pd.read_excel("data/MiscStat.xlsx-20160201.xls")
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160202.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160203.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160204.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160205.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160206.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160207.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160208.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160209.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160210.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160211.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160212.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160213.xls"))
data = data.append(pd.read_excel("data/MiscStat.xlsx-20160214.xls"))

In [3]:
google_data = data[data["PartnerDomain"] == "google"]
google_data.head(1000)
print ""

In [4]:
supersave_data = google_data[google_data["Site"]=="supersaver_se"]
supersave_data.set_index("OrderDateTime")
plt.grid(True)
plt.plot_date(pd.to_datetime(supersave_data.OrderDateTime), supersave_data.MrkCost,"o-")
plt.show()

In [5]:
from statsmodels import tsa
plt.bar(range(len(stat.acf(supersave_data.MrkCost))),stat.acf(supersave_data.MrkCost), width=0.01)
diff = supersave_data.MrkCost.diff()
diff = diff.fillna(0.0)
diff_acf =  stat.pacf(diff)
plt.figure()
plt.ylim(-0.5,0.2)
print diff_acf
plt.bar(range(len(diff_acf)),stat.pacf(diff,method="ols"), width=0.01)
plt.axhline(y = -0.1)
plt.show()

[  1.00000000e+00  -2.56123157e-01  -7.66073235e-02  -1.66964859e-02
  -3.53430416e-03  -7.42759909e-04  -2.89270154e-04  -2.34805053e-04
  -6.08997689e-02   2.69974523e-02   1.89198293e-02   1.67628316e-02
  -1.31192008e-04  -1.57916665e-03  -1.37724123e-03  -1.25483371e-03
  -2.79409542e-03   4.27071723e-03   4.52724895e-04  -1.71443068e-04
  -1.93871623e-03  -9.50326356e-04  -5.53324125e-03   2.37283350e-03
   1.53735081e-03   9.96626923e-04  -1.05918707e-04  -1.88112357e-02
   7.88459963e-03   8.23128293e-03  -1.52650477e-03   1.95912333e-03
   5.36389893e-04   3.58723043e-02  -1.62939184e-01  -9.53923322e-02
  -2.77965188e-02  -7.09099420e-03  -7.13252076e-02   2.86397079e-02
   2.17056605e-02]


In [77]:
training_data = supersave_data.MrkCost.values[0:500]
arima_model = tsa.arima_model.ARIMA(training_data, (2,0,1))
fit_model = arima_model.fit(trend='c')

pred = arima_model.predict(params=fit_model.params,start=0,end=500,dynamic=False)
print pred.size
print supersave_data.OrderDateTime[0:-1].size
plt.plot_date(pd.to_datetime(supersave_data.OrderDateTime[0:500]), supersave_data.MrkCost[0:500],"-")
plt.figure()
#plt.plot_date(pd.to_datetime(supersave_data.OrderDateTime[:]), pred, "r-")
plt.plot(range( len(pred) ), pred, "r-" )
plt.show()

501
669


In [59]:
# import pandas as pd
# import scipy
# import numpy as np
# from scipy import stats
# from matplotlib import pylab as plt
# import category_encoders as ce
# import operator

# # silience the chained-assignment warning
# pd.set_option('chained_assignment',None)

# class BinEncoder():
#     def __init__(self, max_bins=5):
#         """
#         :param data: input data that must be a pandas dataframe
#         :param num_bins: intended num of bins
#         :return:
#         """
#         self.max_bins = max_bins
#         self.actual_bins = 0
#         self.b = np.array([])
#         self.freq = []
#         self.fit_flag = False
#         self.map_tmp = {}
#         self.bin_id_record={}

#     def occurence_count(self, data=pd.Series()):
#         """
#         calculate the frequency for each unique id.
#         :return: the occurrence for each unique id
#         """
#         occ = dict({})
#         for val in data.values:
#             occ[val] = 0
#         for val in data.values:
#             occ[val] += 1
#         return occ

#     def ordering_id_map(self, data=pd.Series()):
#         """
#         Map the original random ID to the ordering ID based on frequency
#         :return: mapped data marked by ordering numbers
#         """
#         if not self.fit_flag:
#             tmp_occurrence = self.occurence_count(data)
#             tmp_sorted_bin = sorted(tmp_occurrence.items(), key=operator.itemgetter(1), reverse=True)
#             tmp_sorted_bin = np.asarray( tmp_sorted_bin )
#             for o,origid in enumerate(tmp_sorted_bin[:,0]):
#                 self.map_tmp[origid] = o
#             print "finished mapping construction......"
#         length = len(self.map_tmp)
#         i= 0
#         for key,u in zip(self.map_tmp.keys(), self.map_tmp.values()):
#             i+= 1.0
#             if i%100 == 0:
#                 print "finished %.5f percentage" %( float(i/length) *100 )

#             row_index = data == key
#             data.loc[row_index] = u

#         print type(data), type(self.map_tmp)
#         diff = set( data.values ) - set(self.map_tmp.values() )
#         print diff
#         if not diff:
#             pass
#         else:
#             print "warning: there are new alien data sent in, please fit with the new data..."
#             for p in list(diff):
#                 row_index = data==p
#                 data.loc[row_index] = np.nan
#         return data

#     def log_n(self, data, n):
#         """
#         log transformation of the input data
#         :param data: input data
#         :param n: n times log taken
#         :return: transformed data
#         """
#         data = np.log(data)
#         # print "%.2f th data result is %s" %(n, str(data))
#         if n <= 1.00:
#             # print "returned... %s" %str(data)
#             return data
#         else:
#             return self.log_n(data + 1, n-1)

#     def bin_id_map(self, p, transformed_id):
#         """
#         Map the ordering ID to bin ID
#         :param p: an array which contains value scope for each bin
#         :param transformed_id: transformed ordering ID
#         :return:  data marked by bin ID
#         """
#         assert  isinstance(transformed_id, pd.Series), "check out your input type..."
#         if not self.fit_flag:
#             a = 0
#             b = 0
#             id_bin = 0
#             for i in range(len(p)-1):
#                 a = b
#                 b = p[i+1]
#                 jd = transformed_id.between(a,b)
#                 if (jd==True).any():
#                     id_bin += 1
#                     transformed_id[jd] = id_bin
#                     self.bin_id_record[id_bin] = [a,b]
#             self.actual_bins = id_bin
#         else:
#             for key, u in zip(self.bin_id_record.keys(),self.bin_id_record.values()):
#                 transformed_id[transformed_id.between(u[0],u[1])] = key
#         return transformed_id


#     def fit(self, data=pd.Series(),show_diag=True):
#         """
#         Automatic binning process, the result is put in the self.data
#         :param show_diag: True for showing the binning histogram
#         :return:
#         """
#         data = self.ordering_id_map(data)
#         # tmp_occurrence = self.occurence_count(data)
#         # tmp_sorted_bin = sorted(tmp_occurrence.items(), key=operator.itemgetter(1), reverse=True)
#         # tmp_sorted_bin = np.asarray( tmp_sorted_bin )
#         # use matplotlib even-spaced binning
#         trans_id = self.log_n(data + 1, 2)
#         self.freq,self.b,p = plt.hist(trans_id , bins=self.max_bins)
#         data = self.bin_id_map(self.b, trans_id)
#         self.fit_flag = True
#         if show_diag:
#             plt.show()

#     def transform(self, data = pd.Series()):
#         """
#         :param data: target data
#         :return: transformed data with previous fit
#         """
#         if not self.fit_flag:
#             raise Exception("You have not fitted your model yet...")
#         data = self.ordering_id_map(data)
#         trans_id = self.log_n(data+1, 2)
#         tran_data = self.bin_id_map(self.b, trans_id)
#         print tran_data
#         tran_data.fillna(self.actual_bins + 1)
#         return tran_data







# if __name__ == "__main__":
#     # path where you put the testing data
#     path = "/Users/gausslee/Documents/programming/etravelipython/ibemingdata/newsql_data20132014n.csv"
#     data = pd.read_csv(path)
#     raw_data = list(data)
#     print "data loaded..................."
#     selection_cols = [ 'OrderDay','SiteID', 'SiteCountryID',
#      'FirstAirSystem',
#      'TravelDays', 'DaysAhead' , 'IsInterCont', 'JouneyType',
#      'NumBounds', 'NumSegs', 'Bound2StartSeg1Bs', 'NumofAd',
#      'NumofCh', 'Numofinf', 'ServDays', 'Net', 'TaxExVat', 'Vat',
#      'MarkupSeg', 'BookType','OriginAirportID', 'OriginCityID','OriginCountryID', 'DestCityID', 'DestCoutID','MarketCostSek']
#     training_data = data[selection_cols]
#     binning =  BinEncoder(max_bins = 20)
#     print "start to bin...."
#     binning.fit(training_data["SiteID"],  False)
#     data = binning.transform(training_data["SiteID"])
#     print data.to_csv("data.csv")
#     alien_data = pd.Series([-9999, 39, -19])
#     alien_data = binning.transform(alien_data)
#     print alien_data